In [2]:
%matplotlib inline
from pandas import DataFrame, read_csv, Series, to_datetime
from matplotlib import pyplot as plt
from os import chdir
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

metadata = DataFrame.from_csv("metadata/GLBRC_MetaT_Metadata.tsv",sep='\t')
metadata['sampling_date'] = to_datetime(metadata.date) #Make date a format python can sort
metadata.drop(['time','air_temp_c', 'day', 'month', 'year', 'weather', 'notes', 'rep', 'date_of_extraction', 'nucleic_acid_type', 
   'replicate_extraction', 'source', 'source_mass', 'extraction_method', 'elution_vol_ul', 'concentration_ng_per_ul', 
   'ratio_260_280', 'conc_ng_per_g_source', 'extracted_by', 'sequencing_date', 'conc_sent_ng_per_ul', 'sequencing_type', 
   'sequencing_facility', 'primers', 'submitted_for_sequencing', 'sequencing_successful', 'duplicate_submitted', 'dup_sequencing_name', 
   'exclude_from_analysis', 'itemID_JGI', 'sampleID_JGI', 'JGI_rawdataname', 'Air_Pressure', 'RH', 'AH', 'Wind_Speed_Mean', 'PAR', 
   'soil_temp_5_cm_bare_avg', 'soil_temp_5_cm_sod_avg', 'Year', 'date', 'pseudorep','MMPRNT_ID','time_zone','longitude', 'country',
   'location','air_temp_max','Air_Temp_Min','latitude','altitude','plot_name', 'soil_name', 'number_cores', 'Air_temp_mean' ,
   'Wind_Direction_Mean','time_numeric','precipitation', 'Solar_Radiation','pH','JGI_taxonOID','JGI_library','SPNL_date','lime_index',
   'P_ppm','barcode','K_ppm', 'Ca_ppm', 'Mg_ppm', 'organic_matter', 'NO3N_ppm', 'NH4_ppm', 'soil_moisture_percent', 'soil_temp_10cm', 
   'plant_name', 'LDMC_mg_per_g', 'nitrogen_percent', 'carbon_percent', 'carbon_per_nitrogen', 'height_mean_cm', 'mass_per_leaf_g',
   'name','plotID','sequence_name','HPCC_path'],axis=1,inplace=True)
metadata = metadata.rename(index=str, columns={'nucleic_acid_name':'name'})

#Sort the metadata for ordering purposes
metadata.sort_values(by=['plant','sampling_date','treatment','name'],inplace=True) #,"Date","treatment","plot_name"])

#Change the Identifier
metadata.set_index('name',inplace=True)

#chdir("/mnt/research/ShadeLab/GLBRC/mapping/metaT")

In [2]:
metadata.head(10)

,plant,treatment,sampling_date
name,,,
G5R1_NF_31MAY2016_LR2,switchgrass,nitrogen free,2016-05-31
G5R3_NF_31MAY2016_LR2,switchgrass,nitrogen free,2016-05-31
G5R4_NF_31MAY2016_LR3,switchgrass,nitrogen free,2016-05-31
G5R1_MAIN_31MAY2016_LR1,switchgrass,standard fertilization,2016-05-31
G5R2_MAIN_31MAY2016_LR1,switchgrass,standard fertilization,2016-05-31
G5R3_MAIN_31MAY2016_LR1,switchgrass,standard fertilization,2016-05-31
G5R4_MAIN_31MAY2016_LR1,switchgrass,standard fertilization,2016-05-31
G5R1_NF_12JUL2016_LR1,switchgrass,nitrogen free,2016-12-07
G5R2_NF_12JUL2016_LR1,switchgrass,nitrogen free,2016-12-07


In [3]:
%%bash
ls

logs
NexteraPE-PE.fa
paired
stats
unpaired


In [6]:
class KO_Mapper:
    def __init__(self):
        self.koToMap = {}
        self.descripts = {}
        self.mapToKO = {}
        self.levelToMap = {}
        self.ko = ""
        self.KEGG_URL = "https://www.kegg.jp/dbget-bin/www_bget?%s"

    def setItem(self, level, mapNum, descrip):
        # print(self.ko, level, mapNum, descrip)
        # Map descriptions
        self.descripts[mapNum] = descrip
        
        # Map to KO
        try: self.mapToKO[mapNum].add(self.ko)
        except: self.mapToKO[mapNum] = set([self.ko])
        
        # KO to level to mapNum
        try: self.koToMap[self.ko][level].add(mapNum)
        except: 
            try: self.koToMap[self.ko][level] = set([mapNum])
            except: self.koToMap[self.ko] = { level:set([mapNum]) }
        
        # Level to map
        try: self.levelToMap[level].add(mapNum)
        except: self.levelToMap[level] = set([mapNum])
                
    def _processKOInfo(self, koText):
        rec = koText.strip().split('\n')
        while 'KEGG Orthology' not in rec[0] and len(rec)>0: rec = rec[1:]
        if len(rec)==0:return
        for line in rec[1:]:
            level = line.count('\xa0')
            if (level == 0) or self.ko in str(line): break #Don't record any ribosome info
            info = line.strip('\xa0')
            bIndex = info.find(" ")
            self.setItem(level,info[:bIndex],info[bIndex+1:])
    
    def mapKO(self,ko):
        siteContent = BeautifulSoup(urlopen(self.KEGG_URL%(ko)).read(),features="lxml")
        self.ko = ko
        found = False
        for i, elm in enumerate(siteContent.find_all("td", {"class": "td41"})):
            if "KEGG Orthology" in str(elm.contents[0]):
                pathway = elm.find("nobr")
                self._processKOInfo(pathway.text)
                found = True
                break
        if not found: 
            for i, elm in enumerate(siteContent.find_all("td", {"class": "td40"})):
                if "KEGG Orthology" in str(elm.contents[0]):
                    pathway = elm.find("nobr")
                    self._processKOInfo(pathway.text)
                    found = True
                    break
        if not found: raise Exception("Unable to find " + self.ko)

In [7]:
from pickle import load
# from KeggAnnotationMapping import KO_Mapper
failed = load(open("pickles/koMapFails.p","rb"))
ko_mapper = load(open("pickles/koMap.p","rb"))

failed

{'K00870', 'K06021', 'K06022', 'K10826', 'K10834', 'K19795', 'K20905'}

In [9]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [22]:
ko_mapper.koToMap['K19795'] = {1:set(['09100']),2:set(['09104']),3:set(['00240'])}

In [44]:
%%bash
pwd
#mv annotations/final.contigs.* mags/
head annotations/KO_Levels.tsv

/mnt/research/ShadeLab/GLBRC
KO_Number	Level1	Level2	Level3
K14681	09100	09105	00250
K04045	09180	09182	03110
K02598	09180	09183	02000
K09011	09100	09101	00660
K09952	09180	09183	02048
K07639	09130	09132	02020
K10715	09130	09132	02020
K00518	09190	09191	99980
K21990	09190	09193	99977


In [43]:
koTableFile = open("annotations/KO_Levels.tsv",'w')

koTableFile.write("\t".join(["KO_Number","Level1","Level2","Level3"])+"\n")
for ko,trace in ko_mapper.koToMap.items():
    if len(trace[1])>1:print(trace[1])
    if len(trace[2])>1:print(trace[2])
    if len(trace[3])>1:print(trace[3])
    trace[1] = list(trace[1])[0]
    trace[2] = list(trace[2])[0]
    trace[3] = list(trace[3])[0]
    koTableFile.write("\t".join([ko,trace[1],trace[2],trace[3]])+"\n")
koTableFile.close()

In [14]:
description = soup_mysite.find("td", {"class": "td41"}) # meta tag description

In [ ]:
from easyFunctions import *

In [54]:
for i, elm in enumerate(soup_mysite.find_all("td", {"class": "td41"})):
#     print(elm.contents[0], "KEGG Orthology" in str(elm.contents[0]))
    if "KEGG Orthology" in str(elm.contents[0]):
        pathway = elm.find("nobr")
#         print(i,pathway.text)
        print(processKOInfo(pathway.text))
#         print(dir(pathway))
        break

1  09120 Genetic Information Processing
2   09122 Translation
3    03010 Ribosome
6     K02907  RP-L30, MRPL30, rpmD; large subunit ribosomal protein L30
1  09180 Brite Hierarchies
2   09182 Protein families: genetic information processing
3    03011 Ribosome
6     K02907  RP-L30, MRPL30, rpmD; large subunit ribosomal protein L30
{}


In [26]:
elm.contents[0]

<div style="width:555px;overflow-x:auto;overflow-y:hidden"><nobr>KEGG Orthology (KO) [BR:<a href="/kegg-bin/get_htext?ko00001+K02907">ko00001</a>]<br/>
 09120 Genetic Information Processing<br/>
  09122 Translation<br/>
   03010 Ribosome<br/>
    K02907  RP-L30, MRPL30, rpmD; large subunit ribosomal protein L30<br/>
 09180 Brite Hierarchies<br/>
  09182 Protein families: genetic information processing<br/>
   03011 Ribosome<br/>
    K02907  RP-L30, MRPL30, rpmD; large subunit ribosomal protein L30<br/>
Ribosome [BR:<a href="/kegg-bin/get_htext?ko03011+K02907">ko03011</a>]<br/>
 Ribosomal Proteins<br/>
  Mitochondria/ Chloroplast<br/>
   Large subunit<br/>
    K02907  RP-L30, MRPL30, rpmD; large subunit ribosomal protein L30<br/>
  Bacteria<br/>
   Large subunit<br/>
    K02907  RP-L30, MRPL30, rpmD; large subunit ribosomal protein L30<br/>
  Archaea<br/>
   Large subunit<br/>
    K02907  RP-L30, MRPL30, rpmD; large subunit ribosomal protein L30<br/>
</nobr></div>

In [1]:
%%bash
ls mapping/metaT/fullAssembly/logs/multiqc_data/

multiqc_bowtie2.txt
multiqc_data.json
multiqc_general_stats.txt
multiqc.log
multiqc_sources.txt
multiqc_trimmomatic.txt


In [6]:
trimreport = read_csv("mapping/metaT/fullAssembly/logs/multiqc_data/multiqc_trimmomatic.txt",sep='\t')
trimreport['surviving'].describe()



count    7.400000e+01
mean     8.713330e+07
std      2.664098e+07
min      8.823170e+06
25%      7.297269e+07
50%      8.448782e+07
75%      1.037760e+08
max      1.492427e+08
Name: surviving, dtype: float64

In [11]:
trimreport['surviving'].median()

84487816.5

In [19]:
alnreport[alnreport['overall_alignment_rate'] == minReadpct]

,Sample,total_reads,paired_total,paired_aligned_none,paired_aligned_one,paired_aligned_multi,paired_aligned_discord_one,paired_aligned_mate_none,paired_aligned_mate_one,paired_aligned_mate_multi,overall_alignment_rate,paired_aligned_mate_multi_halved,paired_aligned_mate_none_halved,paired_aligned_mate_one_halved
51,G5R3_NF_18SEP2017_LR1.stat,69561356,69561356,28955124,18879908,21726324,1077474,42837873,5049560,7867867,69.21,3933933.5,21418936.5,2524780.0


In [18]:
minReadpct = alnreport['overall_alignment_rate'].min()
list(alnreport[alnreport['overall_alignment_rate'] == minReadpct]['total_reads'])[0]*(minReadpct/100.0)

48143414.4876

In [13]:
alnreport['overall_alignment_rate'].mean(),alnreport['overall_alignment_rate'].min(),alnreport['overall_alignment_rate'].max()

(89.03554054054057, 69.21, 98.23)

In [12]:
alnreport = read_csv("mapping/metaT/fullAssembly/logs/multiqc_data/multiqc_bowtie2.txt",sep='\t')
alnreport['overall_alignment_rate']

,Sample,total_reads,paired_total,paired_aligned_none,paired_aligned_one,paired_aligned_multi,paired_aligned_discord_one,paired_aligned_mate_none,paired_aligned_mate_one,paired_aligned_mate_multi,overall_alignment_rate,paired_aligned_mate_multi_halved,paired_aligned_mate_none_halved,paired_aligned_mate_one_halved
0,G5R1_MAIN_05JUN2017_LR1.stat,92085297,92085297,18877880,10809513,62397904,394361,18675589,2746123,15545326,89.86,7772663.0,9337794.5,1373061.5
1,G5R1_MAIN_07AUG2017_LR1.stat,93559623,93559623,12164357,12376618,69018648,500481,7770925,2169461,13387366,95.85,6693683.0,3885462.5,1084730.5
2,G5R1_MAIN_12JUL2016_LR1.stat,103056072,103056072,17492320,11091583,74472169,550060,16394039,2312561,15177920,92.05,7588960.0,8197019.5,1156280.5
3,G5R1_MAIN_12SEP2016_LR1.stat,11458176,11458176,3889944,1937016,5631216,144411,5812184,343133,1335749,74.64,667874.5,2906092.0,171566.5
4,G5R1_MAIN_15MAY2017_LR1.stat,84571087,84571087,9395201,1375153,73800733,37785,3506891,482155,14725786,97.93,7362893.0,1753445.5,241077.5
5,G5R1_MAIN_17JUL2017_LR1.stat,98545553,98545553,30224341,16159488,52161724,933978,32513206,5497128,20570392,83.50,10285196.0,16256603.0,2748564.0
6,G5R1_MAIN_18SEP2017_LR1.stat,75961987,75961987,16157943,18574273,41229771,945065,15127075,3320554,11978127,90.04,5989063.5,7563537.5,1660277.0
7,G5R1_MAIN_26JUN2017_LR1.stat,96920295,96920295,15243383,8397526,73279386,327334,13137031,2347189,14347878,93.22,7173939.0,6568515.5,1173594.5
8,G5R1_MAIN_28AUG2017_LR1.stat,71836493,71836493,22567970,18827620,30440903,922356,30587455,3915706,8788067,78.71,4394033.5,15293727.5,1957853.0
9,G5R1_MAIN_31MAY2016_LR1.stat,125109376,125109376,22558818,11942367,90608191,478645,21703065,2692835,19764446,91.33,9882223.0,10851532.5,1346417.5
